In [9]:
import cv2 as cv 
import numpy as np
import mediapipe as mp 
from eye_contour import *

mp_face_mesh = mp.solutions.face_mesh

In [10]:
# Colors to use (maybe use from another file)
GREEN = (0,255,0)

# left eyes indices
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 

# irises Indices list
LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

In [11]:
cap = cv.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=3,
    refine_landmarks=True,
    min_detection_confidence=0.65,
    min_tracking_confidence=0.65
) as face_mesh:

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1)

        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        mask = np.zeros((img_h, img_w), dtype=np.uint8)

        if results.multi_face_landmarks:
            # print((results.multi_face_landmarks[0]))

            # [print(p.x, p.y, p.z ) for p in results.multi_face_landmarks[0].landmark]
            
            mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) 
            for p in results.multi_face_landmarks[0].landmark])

            mesh_coords = landmarksDetection(frame, results, False)
            
            cv.polylines(frame, [mesh_points[LEFT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            cv.polylines(frame, [mesh_points[RIGHT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)

            cv.circle(frame, center_left, int(l_radius), (180,240,10), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (245,170,20), 1, cv.LINE_AA)

            cv.circle(frame, center_left, 1, (255,0,0), -1, cv.LINE_AA)
            cv.circle(frame, center_right, 1, (255,0,0), -1, cv.LINE_AA)

            # drawing on the mask 
            cv.circle(mask, center_left, int(l_radius), (255,255,255), -1, cv.LINE_AA)
            cv.circle(mask, center_right, int(r_radius), (255,255,255), -1, cv.LINE_AA)

            # -----------------------------------------------------------------------------------
            # Contour 

            # Mask
            left_coords = [mesh_coords[p] for p in LEFT_EYE]
            right_coords = [mesh_coords[p] for p in RIGHT_EYE]
            crop_right, crop_left = eye_contour(frame, right_coords, left_coords)
            
            cv.polylines(frame,  [np.array([mesh_coords[p] for p in LEFT_EYE ], dtype=np.int32)], True, GREEN, 1, cv.LINE_AA)
            cv.polylines(frame,  [np.array([mesh_coords[p] for p in RIGHT_EYE ], dtype=np.int32)], True, GREEN, 1, cv.LINE_AA)
            # -----------------------------------------------------------------------------------
          
        # cv.imshow('Mask_pupil', mask)     
        cv.imshow('img', frame)
        key = cv.waitKey(1)
        if key == ord('q'): # off pressing "q"
            break

cap.release()
cv.destroyAllWindows()